# Data Processing  Phase 1

This step is the first step in model developing.

It include initial steps before the data anlaysis such as:

1. Change features names.
2. Change datatypes.
3. Remove unwanted features.
4. Imputing missing values.
5. Deal with duplications.
6. Deal with logical validations.

Next step is Data Analysis.

# Import

Import all the needed modules

In [1]:
import os
import json
import re
import pandas as pd
import numpy as np
from dask import dataframe as dd
from dask.distributed import Client, LocalCluster
import warnings

# Init

In [2]:
# ignore warning
warnings.filterwarnings("ignore")

# pandas columns setting
pd.set_option('display.max_columns', 30)

# disable warning
import warnings
warnings.filterwarnings('ignore')

# notebook width
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:80% !important; }</style>"))

# dask cluster
cluster = LocalCluster(n_workers=2, threads_per_worker=1, processes=False)
client = Client(cluster)

In [3]:
client

Client Scheduler: inproc://172.17.0.2/8575/1 Dashboard: http://172.17.0.2:8787/status,Cluster Workers: 2 Cores: 2 Memory: 8.25 GB


In [4]:
processing_steps = []

# Load data

In [5]:
file_path = '/workspaces/product-recommendation-system/product-recommendation-system-dataset/raw-data'

# read data
df = dd.read_parquet(file_path)

In [6]:
# data first rows
df.head()

,event_time,event_type,product_id,category_id,category_code,brand,price,user_id,user_session
index,,,,,,,,,
0,2019-11-01 00:00:00 UTC,view,1003461,2053013555631882655,electronics.smartphone,xiaomi,489.07,520088904,4d3b30da-a5e4-49df-b1a8-ba5943f1dd33
1,2019-11-01 00:00:00 UTC,view,5000088,2053013566100866035,appliances.sewing_machine,janome,293.65,530496790,8e5f4f83-366c-4f70-860e-ca7417414283
2,2019-11-01 00:00:01 UTC,view,17302664,2053013553853497655,None,creed,28.31,561587266,755422e7-9040-477b-9bd2-6a6e8fd97387
3,2019-11-01 00:00:01 UTC,view,3601530,2053013563810775923,appliances.kitchen.washer,lg,712.87,518085591,3bfb58cd-7892-48cc-8020-2f17e6de6e7f
4,2019-11-01 00:00:01 UTC,view,1004775,2053013555631882655,electronics.smartphone,xiaomi,183.27,558856683,313628f1-68b8-460d-84f6-cec7a8796ef2


In [7]:
# get info on data types, size and missing values
df.info()

<class 'dask.dataframe.core.DataFrame'>
Columns: 9 entries, event_time to user_session
dtypes: object(5), float64(1), int64(3)

In [8]:
df_shape = client.compute(df.shape[0]).result(), df.shape[1]
df_shape

(109950743, 9)

In [9]:
# # change data columns names to PEP 
# df.columns = [col.lower() if '_' in col else '_'.join(re.findall('[A-Z][^A-Z]*', col)).lower() for  col in df.columns]

In [10]:
# # rename jobactualactivedays
# df.rename(index=str, columns={'original_col': 'new_col'}, inplace=True)

# Data Invalidations

Data Invalidations is being made by the next steps:
1. Search for uneeded features.
2. Data types correctness.
3. Searching missing values.
4. Duplications in data set.
5. Logical validation.

In [11]:
# new dataframe columns names info
df.dtypes

event_time        object
event_type        object
product_id         int64
category_id        int64
category_code     object
brand             object
price            float64
user_id            int64
user_session      object
dtype: object

## remove features

Some features were extracted from the database and has no need in the next model development steps. 
Reducing features number has a great contribution on model development success.  

In [12]:
# # drop undeeded features
# df.drop(['col1', 'col2'], axis=1, inplace=True)

## Data Types

Covert to appropriate data types.

In [6]:
# datetime
def fix_data_types(dataframe):
    dataframe.event_time = dd.to_datetime(dataframe.event_time)
    
    return dataframe

processing_steps.append(fix_data_types)

## Missing values

Missing values are something common in every dataset. 
The following steps can be applied:
1. Too many missing values in some features - drop columns
2. Some rows has many missing values - drop rows.
3. Inconsistancy inmissing values - fill values with soem strategy: constant value, means, quantiles or advanced imputing strategy.

In [14]:
# calculate missing values of each feature
# cols_counts = df.count(axis=0).compute()
cols_null_counts = client.compute(df.isnull().sum(axis=0)).result()

print('Null report:')
display(cols_null_counts)
display((cols_null_counts/df_shape[0]).apply(lambda x: '{:.4%}'.format(x)))

Null report:


event_time              0
event_type              0
product_id              0
category_id             0
category_code    35413780
brand            15341158
price                   0
user_id                 0
user_session           12
dtype: int64

event_time        0.0000%
event_type        0.0000%
product_id        0.0000%
category_id       0.0000%
category_code    32.2088%
brand            13.9528%
price             0.0000%
user_id           0.0000%
user_session      0.0000%
dtype: object

In [15]:
df[df.category_code.isnull()].head()

,event_time,event_type,product_id,category_id,category_code,brand,price,user_id,user_session
index,,,,,,,,,
2,2019-11-01 00:00:01 UTC,view,17302664,2053013553853497655,None,creed,28.31,561587266,755422e7-9040-477b-9bd2-6a6e8fd97387
7,2019-11-01 00:00:02 UTC,view,15900065,2053013558190408249,None,rondell,30.86,518574284,5e6ef132-4d7c-4730-8c7f-85aa4082588f
8,2019-11-01 00:00:02 UTC,view,12708937,2053013553559896355,None,michelin,72.72,532364121,0a899268-31eb-46de-898d-09b2da950b24
13,2019-11-01 00:00:04 UTC,view,34600011,2060981320581906480,None,None,20.54,512416379,4dfe2c67-e537-4dc2-ae69-0b65263db091
15,2019-11-01 00:00:06 UTC,view,24900193,2053013562183385881,None,None,1.09,512651494,f603c815-f51a-46fe-9404-cb586e35edaf


In [16]:
df[df.brand.isnull()].head()

,event_time,event_type,product_id,category_id,category_code,brand,price,user_id,user_session
index,,,,,,,,,
10,2019-11-01 00:00:03 UTC,view,17200570,2053013559792632471,furniture.living_room.sofa,None,437.33,518780843,aa806835-b14c-45af-9538-cd4d1849be3e
11,2019-11-01 00:00:03 UTC,view,2701517,2053013563911439225,appliances.kitchen.refrigerators,None,155.11,518427361,c89b0d96-247f-4044-9c91-bb5f38c6af9b
12,2019-11-01 00:00:03 UTC,view,16700260,2053013559901684381,furniture.kitchen.chair,None,31.64,566255262,173d7b72-1db7-4638-8207-be8148bf3c9f
13,2019-11-01 00:00:04 UTC,view,34600011,2060981320581906480,None,None,20.54,512416379,4dfe2c67-e537-4dc2-ae69-0b65263db091
15,2019-11-01 00:00:06 UTC,view,24900193,2053013562183385881,None,None,1.09,512651494,f603c815-f51a-46fe-9404-cb586e35edaf


In [17]:
# calculate missing values of category features (empty string)
df_cat_columns = df.select_dtypes(include='object')
col_counts_empty_string = client.compute((df_cat_columns=='').sum(axis=0)).result()

print('Categories column empty string report:')
display(col_counts_empty_string)
display((col_counts_empty_string/df_shape[0]).apply(lambda x: '{:.4%}'.format(x)))

Categories column empty string report:


event_time       0
event_type       0
category_code    0
brand            0
user_session     0
dtype: int64

event_time       0.0000%
event_type       0.0000%
category_code    0.0000%
brand            0.0000%
user_session     0.0000%
dtype: object

__Fast Imputing__

First we will check how to handle missing user_session. As the amount of missing sessions is very small, we will probably drop those samples.

In [18]:
missing_session = client.compute(df[df.user_session.isnull()]).result()
missing_session

,event_time,event_type,product_id,category_id,category_code,brand,price,user_id,user_session
index,,,,,,,,,
357562,2019-11-09 15:32:27 UTC,cart,19700004,2053013559104766575,None,kabrita,37.77,539704497,None
98682,2019-11-09 17:15:24 UTC,cart,1005083,2053013555631882655,electronics.smartphone,honor,566.27,568843390,None
300369,2019-11-13 04:02:03 UTC,cart,4804008,2053013554658804075,electronics.audio.headphone,bluedio,97.81,570411102,None
149937,2019-11-13 07:18:35 UTC,cart,1004767,2053013555631882655,electronics.smartphone,samsung,243.51,570878749,None
345764,2019-11-23 12:53:50 UTC,cart,7600528,2053013552821698803,None,tp-link,16.73,575357602,None
441505,2019-11-25 05:04:42 UTC,cart,1802104,2053013554415534427,electronics.video.tv,arg,360.09,573722572,None
128735,2019-11-25 07:03:38 UTC,cart,21403753,2053013561579406073,electronics.clocks,None,181.47,576301354,None
350718,2019-11-26 07:48:12 UTC,cart,4804718,2053013554658804075,electronics.audio.headphone,apple,334.58,576935861,None
471554,2019-11-27 07:02:20 UTC,cart,12719553,2053013553559896355,None,None,55.60,577167045,None


There is no patterns with this column missing values. Let's check one of the users history.

In [19]:
client.compute(df[df.user_id==539704497]).result()

,event_time,event_type,product_id,category_id,category_code,brand,price,user_id,user_session
index,,,,,,,,,
356804,2019-11-09 15:32:06 UTC,view,19700004,2053013559104766575,None,kabrita,37.77,539704497,e82ecde4-5c22-49e6-84cc-d63def4ff7d3
357562,2019-11-09 15:32:27 UTC,cart,19700004,2053013559104766575,None,kabrita,37.77,539704497,None
357725,2019-11-09 15:32:32 UTC,view,19700004,2053013559104766575,None,kabrita,37.77,539704497,e1e253ec-b564-48a8-8ff5-4e86bacb49f3
370418,2019-11-09 15:38:25 UTC,view,19700004,2053013559104766575,None,kabrita,37.77,539704497,e1e253ec-b564-48a8-8ff5-4e86bacb49f3
370730,2019-11-09 15:38:34 UTC,cart,19700004,2053013559104766575,None,kabrita,37.77,539704497,e1e253ec-b564-48a8-8ff5-4e86bacb49f3
371092,2019-11-09 15:38:44 UTC,view,19700004,2053013559104766575,None,kabrita,37.77,539704497,e1e253ec-b564-48a8-8ff5-4e86bacb49f3
76769,2019-11-23 09:45:22 UTC,view,4300011,2053013552385491165,None,philips,118.38,539704497,7c37c229-d95e-4685-b634-b4427e1d331e
78718,2019-11-23 09:46:41 UTC,view,4300011,2053013552385491165,None,philips,118.38,539704497,7c37c229-d95e-4685-b634-b4427e1d331e
78938,2019-11-23 09:46:50 UTC,view,4300011,2053013552385491165,None,philips,118.38,539704497,7c37c229-d95e-4685-b634-b4427e1d331e


In [7]:
def remove_nulls(dataframe):
    dataframe = dataframe.dropna(subset=['user_session'])
    
    return dataframe

processing_steps.append(remove_nulls)

In [8]:
def impute_nulls_as_empty_strings(dataframe):
    # category code
    dataframe.category_code = dataframe.category_code.fillna('')
    
    # brand
    dataframe.brand = dataframe.brand.fillna('')
    
    return dataframe
    
processing_steps.append(impute_nulls_as_empty_strings)

As there is no pattern here also, this missing value will be dropped.

__Complex Imputing__

In [22]:
# some complex null values imputing, such as means or quantiles values.
# knn imputing

## Duplications

There are two possible duplications types:
1. Entire row is duplicated.
2. Duplication according to some business logic (atomic features).

Duplication from type 1 should be dropped.

Duplication from type 2 should be investigated for their origin.

__Duplicated rows__

In [9]:
def drop_duplications(dataframe):
    dataframe = dataframe.drop_duplicates()
    
    return dataframe

processing_steps.append(drop_duplications)

__Atomic features duplications__

In [24]:
# no atomic features to check


## Logical validation

The next logical rules will be tested:

1. Negative values.
2. Check uniques values.
3. Domain knowledge values.


__bla bla__

In [25]:
# some logical rule


__Negative values__

In [26]:
# columns with negative values
cols_negative_values_count = client.compute((df.price <= 0).sum()).result()
cols_negative_values_count

256761

In [27]:
df_negative_pricing = df[df.price <= 0]

df_negative_pricing = client.compute(df_negative_pricing).result()

In [28]:
df_negative_pricing.head()

,event_time,event_type,product_id,category_id,category_code,brand,price,user_id,user_session
index,,,,,,,,,
6258,2019-11-01 00:38:01 UTC,view,33100000,2058719826188173878,None,None,0.0,546996930,969ea68f-a919-4d32-8925-b709d87c539c
7245,2019-11-01 00:42:51 UTC,view,33100000,2058719826188173878,None,None,0.0,546996930,b1ab3863-bbf5-4370-a3d8-bfa70580dd09
12743,2019-11-01 01:07:15 UTC,view,12720812,2053013553559896355,None,None,0.0,516269492,9bf68f2a-fd78-4b19-97ac-7d838c82eb4c
12908,2019-11-01 01:07:58 UTC,view,12720812,2053013553559896355,None,None,0.0,516269492,9bf68f2a-fd78-4b19-97ac-7d838c82eb4c
13503,2019-11-01 01:26:19 UTC,view,38900075,2085718636156158307,None,None,0.0,539587206,4d91487b-2b8d-41e3-9328-f8ab325a95cf


In [29]:
df_negative_pricing[df_negative_pricing.price<0]

,event_time,event_type,product_id,category_id,category_code,brand,price,user_id,user_session
index,,,,,,,,,


In [10]:
def remove_0_pricing(dataframe):
    dataframe = dataframe[dataframe.price>0]
    
    return dataframe

processing_steps.append(remove_0_pricing)

__Unique values__

In [31]:
# unique event type
unique_events = df.event_type.unique()

unique_events = client.compute(unique_events).result()

unique_events

0        view
1        cart
2    purchase
Name: event_type, dtype: object

In [32]:
# unique event type
unique_category_code = df.category_code.unique()

unique_category_code = client.compute(unique_category_code).result()

unique_category_code.head()

0       electronics.smartphone
1    appliances.sewing_machine
2                         None
3    appliances.kitchen.washer
4           computers.notebook
Name: category_code, dtype: object

In [33]:
unique_category_code.shape

(130,)

# Quick EDA

In [34]:
df.dtypes

event_time        object
event_type        object
product_id         int64
category_id        int64
category_code     object
brand             object
price            float64
user_id            int64
user_session      object
dtype: object

## Continuous data

In [35]:
# numeric columns
numerics_cols = ['price']

In [36]:
client.compute(df[numerics_cols].describe()).result()

,price
count,1.099507e+08
mean,2.916348e+02
std,3.566800e+02
min,0.000000e+00
25%,9.189000e+01
50%,2.049400e+02
75%,4.581000e+02
max,2.574070e+03


## Categorical data

In [37]:
cat_cols = ['event_type', 'category_code', 'brand']

In [38]:
client.compute(df[cat_cols].describe(include='object')).result()

distributed.worker - WARNING - Memory use is high but worker has no data to store to disk.  Perhaps some other process is leaking memory?  Process memory: 2.75 GB -- Worker memory limit: 4.13 GB
distributed.worker - WARNING - Memory use is high but worker has no data to store to disk.  Perhaps some other process is leaking memory?  Process memory: 2.68 GB -- Worker memory limit: 4.13 GB
distributed.worker - WARNING - Memory use is high but worker has no data to store to disk.  Perhaps some other process is leaking memory?  Process memory: 2.68 GB -- Worker memory limit: 4.13 GB
distributed.worker - WARNING - Memory use is high but worker has no data to store to disk.  Perhaps some other process is leaking memory?  Process memory: 2.81 GB -- Worker memory limit: 4.13 GB
distributed.worker - WARNING - Memory use is high but worker has no data to store to disk.  Perhaps some other process is leaking memory?  Process memory: 2.81 GB -- Worker memory limit: 4.13 GB
distributed.worker - WARN

,event_type,category_code,brand
unique,3,129,4302
count,109950743,74536963,94609585
top,view,electronics.smartphone,samsung
freq,104335509,27882231,13172020


## Datetime data

In [41]:
event_time_describe = client.compute(dd.to_datetime(df.event_time).describe(include='datetime')).result()

distributed.worker - WARNING - Memory use is high but worker has no data to store to disk.  Perhaps some other process is leaking memory?  Process memory: 2.90 GB -- Worker memory limit: 4.13 GB
distributed.worker - WARNING - Memory use is high but worker has no data to store to disk.  Perhaps some other process is leaking memory?  Process memory: 2.90 GB -- Worker memory limit: 4.13 GB
distributed.worker - WARNING - Memory use is high but worker has no data to store to disk.  Perhaps some other process is leaking memory?  Process memory: 2.96 GB -- Worker memory limit: 4.13 GB
distributed.worker - WARNING - Memory use is high but worker has no data to store to disk.  Perhaps some other process is leaking memory?  Process memory: 2.96 GB -- Worker memory limit: 4.13 GB
distributed.worker - WARNING - Memory use is high but worker has no data to store to disk.  Perhaps some other process is leaking memory?  Process memory: 3.02 GB -- Worker memory limit: 4.13 GB
distributed.worker - WARN

distributed.worker - WARNING - Memory use is high but worker has no data to store to disk.  Perhaps some other process is leaking memory?  Process memory: 2.89 GB -- Worker memory limit: 4.13 GB
distributed.worker - WARNING - Memory use is high but worker has no data to store to disk.  Perhaps some other process is leaking memory?  Process memory: 2.89 GB -- Worker memory limit: 4.13 GB
distributed.worker - WARNING - Memory use is high but worker has no data to store to disk.  Perhaps some other process is leaking memory?  Process memory: 2.89 GB -- Worker memory limit: 4.13 GB
distributed.worker - WARNING - Memory use is high but worker has no data to store to disk.  Perhaps some other process is leaking memory?  Process memory: 2.89 GB -- Worker memory limit: 4.13 GB
distributed.worker - WARNING - Memory use is high but worker has no data to store to disk.  Perhaps some other process is leaking memory?  Process memory: 2.94 GB -- Worker memory limit: 4.13 GB
distributed.worker - WARN

distributed.worker - WARNING - Memory use is high but worker has no data to store to disk.  Perhaps some other process is leaking memory?  Process memory: 2.93 GB -- Worker memory limit: 4.13 GB
distributed.worker - WARNING - Memory use is high but worker has no data to store to disk.  Perhaps some other process is leaking memory?  Process memory: 2.93 GB -- Worker memory limit: 4.13 GB
distributed.worker - WARNING - Memory use is high but worker has no data to store to disk.  Perhaps some other process is leaking memory?  Process memory: 2.93 GB -- Worker memory limit: 4.13 GB
distributed.worker - WARNING - Memory use is high but worker has no data to store to disk.  Perhaps some other process is leaking memory?  Process memory: 2.92 GB -- Worker memory limit: 4.13 GB
distributed.worker - WARNING - Memory use is high but worker has no data to store to disk.  Perhaps some other process is leaking memory?  Process memory: 2.92 GB -- Worker memory limit: 4.13 GB
distributed.worker - WARN

distributed.worker - WARNING - Memory use is high but worker has no data to store to disk.  Perhaps some other process is leaking memory?  Process memory: 2.89 GB -- Worker memory limit: 4.13 GB
distributed.worker - WARNING - Memory use is high but worker has no data to store to disk.  Perhaps some other process is leaking memory?  Process memory: 2.89 GB -- Worker memory limit: 4.13 GB
distributed.worker - WARNING - Memory use is high but worker has no data to store to disk.  Perhaps some other process is leaking memory?  Process memory: 2.93 GB -- Worker memory limit: 4.13 GB
distributed.worker - WARNING - Memory use is high but worker has no data to store to disk.  Perhaps some other process is leaking memory?  Process memory: 2.93 GB -- Worker memory limit: 4.13 GB
distributed.worker - WARNING - Memory use is high but worker has no data to store to disk.  Perhaps some other process is leaking memory?  Process memory: 2.89 GB -- Worker memory limit: 4.13 GB
distributed.worker - WARN

distributed.worker - WARNING - Memory use is high but worker has no data to store to disk.  Perhaps some other process is leaking memory?  Process memory: 2.93 GB -- Worker memory limit: 4.13 GB
distributed.worker - WARNING - Memory use is high but worker has no data to store to disk.  Perhaps some other process is leaking memory?  Process memory: 2.93 GB -- Worker memory limit: 4.13 GB
distributed.worker - WARNING - Memory use is high but worker has no data to store to disk.  Perhaps some other process is leaking memory?  Process memory: 2.92 GB -- Worker memory limit: 4.13 GB
distributed.worker - WARNING - Memory use is high but worker has no data to store to disk.  Perhaps some other process is leaking memory?  Process memory: 2.92 GB -- Worker memory limit: 4.13 GB
distributed.worker - WARNING - Memory use is high but worker has no data to store to disk.  Perhaps some other process is leaking memory?  Process memory: 2.92 GB -- Worker memory limit: 4.13 GB
distributed.worker - WARN

...

distributed.worker - WARNING - Memory use is high but worker has no data to store to disk.  Perhaps some other process is leaking memory?  Process memory: 2.92 GB -- Worker memory limit: 4.13 GB
distributed.worker - WARNING - Memory use is high but worker has no data to store to disk.  Perhaps some other process is leaking memory?  Process memory: 2.92 GB -- Worker memory limit: 4.13 GB
distributed.worker - WARNING - Memory use is high but worker has no data to store to disk.  Perhaps some other process is leaking memory?  Process memory: 2.92 GB -- Worker memory limit: 4.13 GB
distributed.worker - WARNING - Memory use is high but worker has no data to store to disk.  Perhaps some other process is leaking memory?  Process memory: 2.92 GB -- Worker memory limit: 4.13 GB
distributed.worker - WARNING - Memory use is high but worker has no data to store to disk.  Perhaps some other process is leaking memory?  Process memory: 2.92 GB -- Worker memory limit: 4.13 GB
distributed.worker - WARN

distributed.worker - WARNING - Memory use is high but worker has no data to store to disk.  Perhaps some other process is leaking memory?  Process memory: 2.92 GB -- Worker memory limit: 4.13 GB
distributed.worker - WARNING - Memory use is high but worker has no data to store to disk.  Perhaps some other process is leaking memory?  Process memory: 2.92 GB -- Worker memory limit: 4.13 GB
distributed.worker - WARNING - Memory use is high but worker has no data to store to disk.  Perhaps some other process is leaking memory?  Process memory: 2.92 GB -- Worker memory limit: 4.13 GB
distributed.worker - WARNING - Memory use is high but worker has no data to store to disk.  Perhaps some other process is leaking memory?  Process memory: 2.92 GB -- Worker memory limit: 4.13 GB
distributed.worker - WARNING - Memory use is high but worker has no data to store to disk.  Perhaps some other process is leaking memory?  Process memory: 2.92 GB -- Worker memory limit: 4.13 GB
distributed.worker - WARN

distributed.worker - WARNING - Memory use is high but worker has no data to store to disk.  Perhaps some other process is leaking memory?  Process memory: 2.92 GB -- Worker memory limit: 4.13 GB
distributed.worker - WARNING - Memory use is high but worker has no data to store to disk.  Perhaps some other process is leaking memory?  Process memory: 2.92 GB -- Worker memory limit: 4.13 GB
distributed.worker - WARNING - Memory use is high but worker has no data to store to disk.  Perhaps some other process is leaking memory?  Process memory: 2.92 GB -- Worker memory limit: 4.13 GB
distributed.worker - WARNING - Memory use is high but worker has no data to store to disk.  Perhaps some other process is leaking memory?  Process memory: 2.92 GB -- Worker memory limit: 4.13 GB
distributed.worker - WARNING - Memory use is high but worker has no data to store to disk.  Perhaps some other process is leaking memory?  Process memory: 2.92 GB -- Worker memory limit: 4.13 GB
distributed.worker - WARN

distributed.worker - WARNING - Memory use is high but worker has no data to store to disk.  Perhaps some other process is leaking memory?  Process memory: 2.92 GB -- Worker memory limit: 4.13 GB
distributed.worker - WARNING - Memory use is high but worker has no data to store to disk.  Perhaps some other process is leaking memory?  Process memory: 2.92 GB -- Worker memory limit: 4.13 GB
distributed.worker - WARNING - Memory use is high but worker has no data to store to disk.  Perhaps some other process is leaking memory?  Process memory: 2.92 GB -- Worker memory limit: 4.13 GB
distributed.worker - WARNING - Memory use is high but worker has no data to store to disk.  Perhaps some other process is leaking memory?  Process memory: 2.92 GB -- Worker memory limit: 4.13 GB
distributed.worker - WARNING - Memory use is high but worker has no data to store to disk.  Perhaps some other process is leaking memory?  Process memory: 2.92 GB -- Worker memory limit: 4.13 GB
distributed.worker - WARN

distributed.worker - WARNING - Memory use is high but worker has no data to store to disk.  Perhaps some other process is leaking memory?  Process memory: 2.92 GB -- Worker memory limit: 4.13 GB
distributed.worker - WARNING - Memory use is high but worker has no data to store to disk.  Perhaps some other process is leaking memory?  Process memory: 2.92 GB -- Worker memory limit: 4.13 GB
distributed.worker - WARNING - Memory use is high but worker has no data to store to disk.  Perhaps some other process is leaking memory?  Process memory: 2.92 GB -- Worker memory limit: 4.13 GB
distributed.worker - WARNING - Memory use is high but worker has no data to store to disk.  Perhaps some other process is leaking memory?  Process memory: 2.92 GB -- Worker memory limit: 4.13 GB
distributed.worker - WARNING - Memory use is high but worker has no data to store to disk.  Perhaps some other process is leaking memory?  Process memory: 2.92 GB -- Worker memory limit: 4.13 GB
distributed.worker - WARN

# Processing

In [11]:
for step in processing_steps:
    df = step(df)

In [ ]:
df = df.set_index('event_time')

distributed.worker - WARNING - Memory use is high but worker has no data to store to disk.  Perhaps some other process is leaking memory?  Process memory: 3.03 GB -- Worker memory limit: 4.13 GB
distributed.worker - WARNING - Memory use is high but worker has no data to store to disk.  Perhaps some other process is leaking memory?  Process memory: 3.11 GB -- Worker memory limit: 4.13 GB
distributed.worker - WARNING - Memory use is high but worker has no data to store to disk.  Perhaps some other process is leaking memory?  Process memory: 3.66 GB -- Worker memory limit: 4.13 GB
distributed.worker - WARNING - Worker is at 88% memory usage. Pausing worker.  Process memory: 3.66 GB -- Worker memory limit: 4.13 GB
distributed.worker - WARNING - Memory use is high but worker has no data to store to disk.  Perhaps some other process is leaking memory?  Process memory: 3.66 GB -- Worker memory limit: 4.13 GB
distributed.worker - WARNING - Memory use is high but worker has no data to store to 

distributed.worker - WARNING - Memory use is high but worker has no data to store to disk.  Perhaps some other process is leaking memory?  Process memory: 4.11 GB -- Worker memory limit: 4.13 GB
distributed.worker - WARNING - Memory use is high but worker has no data to store to disk.  Perhaps some other process is leaking memory?  Process memory: 4.33 GB -- Worker memory limit: 4.13 GB
distributed.worker - WARNING - Memory use is high but worker has no data to store to disk.  Perhaps some other process is leaking memory?  Process memory: 4.25 GB -- Worker memory limit: 4.13 GB
distributed.worker - WARNING - Memory use is high but worker has no data to store to disk.  Perhaps some other process is leaking memory?  Process memory: 4.25 GB -- Worker memory limit: 4.13 GB
distributed.worker - WARNING - Memory use is high but worker has no data to store to disk.  Perhaps some other process is leaking memory?  Process memory: 3.87 GB -- Worker memory limit: 4.13 GB
distributed.worker - WARN

distributed.worker - WARNING - Memory use is high but worker has no data to store to disk.  Perhaps some other process is leaking memory?  Process memory: 3.67 GB -- Worker memory limit: 4.13 GB
distributed.worker - WARNING - Worker is at 88% memory usage. Pausing worker.  Process memory: 3.67 GB -- Worker memory limit: 4.13 GB
distributed.worker - WARNING - Worker is at 93% memory usage. Pausing worker.  Process memory: 3.86 GB -- Worker memory limit: 4.13 GB
distributed.worker - WARNING - Memory use is high but worker has no data to store to disk.  Perhaps some other process is leaking memory?  Process memory: 3.86 GB -- Worker memory limit: 4.13 GB
distributed.worker - WARNING - Worker is at 76% memory usage. Resuming worker. Process memory: 3.14 GB -- Worker memory limit: 4.13 GB
distributed.worker - WARNING - Memory use is high but worker has no data to store to disk.  Perhaps some other process is leaking memory?  Process memory: 3.21 GB -- Worker memory limit: 4.13 GB
distribute

distributed.worker - WARNING - Memory use is high but worker has no data to store to disk.  Perhaps some other process is leaking memory?  Process memory: 3.91 GB -- Worker memory limit: 4.13 GB
distributed.worker - WARNING - Memory use is high but worker has no data to store to disk.  Perhaps some other process is leaking memory?  Process memory: 3.77 GB -- Worker memory limit: 4.13 GB
distributed.worker - WARNING - Worker is at 91% memory usage. Pausing worker.  Process memory: 3.77 GB -- Worker memory limit: 4.13 GB
distributed.worker - WARNING - Memory use is high but worker has no data to store to disk.  Perhaps some other process is leaking memory?  Process memory: 3.77 GB -- Worker memory limit: 4.13 GB
distributed.worker - WARNING - Memory use is high but worker has no data to store to disk.  Perhaps some other process is leaking memory?  Process memory: 3.77 GB -- Worker memory limit: 4.13 GB
distributed.worker - WARNING - Memory use is high but worker has no data to store to 

distributed.worker - WARNING - Memory use is high but worker has no data to store to disk.  Perhaps some other process is leaking memory?  Process memory: 4.00 GB -- Worker memory limit: 4.13 GB
distributed.worker - WARNING - Memory use is high but worker has no data to store to disk.  Perhaps some other process is leaking memory?  Process memory: 3.94 GB -- Worker memory limit: 4.13 GB
distributed.worker - WARNING - Memory use is high but worker has no data to store to disk.  Perhaps some other process is leaking memory?  Process memory: 4.16 GB -- Worker memory limit: 4.13 GB
distributed.worker - WARNING - Memory use is high but worker has no data to store to disk.  Perhaps some other process is leaking memory?  Process memory: 4.16 GB -- Worker memory limit: 4.13 GB
distributed.worker - WARNING - Memory use is high but worker has no data to store to disk.  Perhaps some other process is leaking memory?  Process memory: 4.32 GB -- Worker memory limit: 4.13 GB
distributed.worker - WARN

distributed.worker - WARNING - Memory use is high but worker has no data to store to disk.  Perhaps some other process is leaking memory?  Process memory: 4.30 GB -- Worker memory limit: 4.13 GB
distributed.worker - WARNING - Memory use is high but worker has no data to store to disk.  Perhaps some other process is leaking memory?  Process memory: 4.30 GB -- Worker memory limit: 4.13 GB
distributed.worker - WARNING - Memory use is high but worker has no data to store to disk.  Perhaps some other process is leaking memory?  Process memory: 4.31 GB -- Worker memory limit: 4.13 GB
distributed.worker - WARNING - Memory use is high but worker has no data to store to disk.  Perhaps some other process is leaking memory?  Process memory: 4.46 GB -- Worker memory limit: 4.13 GB
distributed.worker - WARNING - Memory use is high but worker has no data to store to disk.  Perhaps some other process is leaking memory?  Process memory: 4.46 GB -- Worker memory limit: 4.13 GB
distributed.worker - WARN

# Save Processed data

In [ ]:
# save path
save_path = '/workspaces/product-recommendation-system/product-recommendation-system-dataset/processed-data'

# save
df.to_parquet(save_path, write_metadata_file=False)